In [1]:
import os
import polars as pl
import altair as alt
import warnings

pl.Config(tbl_rows=100)

df = pl.read_parquet(os.path.join("data","fdb.parquet"))
vanocni = pl.read_json(os.path.join("data","vanocni_filmy_kompletni_metadata.json"))

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings("ignore")

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [2]:
vanocni_filmy = vanocni.select(pl.col("nazev")).to_series().to_list()
vanocni_filmy[0:10]

['Anděl Páně',
 'Ať přiletí čáp, královno!',
 'Bejvalek se nezbavíš',
 'Bohouš',
 'Byl jednou jeden král...',
 'Cesta do Rokycan',
 'Chobotnice z II. patra',
 'Co takhle svatba, princi?',
 'Císařův pekař - Pekařův císař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert']

In [3]:
vanocni_filmy.append("Císařův pekař")
vanocni_filmy.append("Pekařův císař")
vanocni_filmy.remove("Bejvalek se nezbavíš")

In [4]:
# vanocni_filmy.append("Pelíšky")

In [5]:
df.filter(
    pl.col("kanal").is_in(["ČT1","ČT2","NOVA","Prima"])
).filter(
    pl.col("nazev").is_in(vanocni_filmy)
)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2011-01-01 09:40:00,"""Láska rohatá""","""26.12.2024 v 14:05 (ČT1)""","""Další díl/opakování:26.12.2024 v 14:05 (ČT1)""","""film/laska-rohata/56863"""
"""NOVA""",2017-06-04 18:20:00,"""Pekařův císař""","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025 v 16:20 (ČT1)""","""film/cisaruv-pekar-a-pekaruv-cisar/5136"""
"""Prima""",2016-12-23 20:15:00,"""S tebou mě baví svět""",null,"""Rodinná komedie ČR (1982). I letos vyrážejí na tajemnou expedici YDYKSEB. Tradiční pánskou jízdu jim však manželky tentokrát pěkně osladí, když s nimi…","""film/s-tebou-me-bavi-svet/18350"""
"""Prima""",2020-12-27 20:15:00,"""Jak se budí princezny""","""29.12.2024 v 10:00 (JOJ+)29.12.2024 v 14:15 (NOVA)30.12.2024 v 10:45 (Nova Cinema)""","""Další díl/opakování:29.12.2024 v 10:00 (JOJ+)29.12.2024 v 14:15 (NOVA)30.12.2024 v 10:45 (Nova Cinema)""","""film/jak-se-budi-princezny/9284"""
"""ČT1""",2016-08-18 03:55:00,"""Světáci""",null,"""Soutěž (2007). Zábavná soutěžní hra o tom, jak si podmanit velký svět. Připravili: M. Przebinda, M. Najbrt, K. Hurych a P. Čepický""",null
"""NOVA""",2022-08-14 09:35:00,"""Princezna ze mlejna""","""29.03.2025 v 07:45 (Markíza)""","""Další díl/opakování:29.03.2025 v 07:45 (Markíza)""","""film/princezna-ze-mlejna-2/16963"""
"""ČT1""",2023-04-09 15:55:00,"""Císařův pekař""","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025 v 16:20 (ČT1)""","""film/cisaruv-pekar-a-pekaruv-cisar/5136"""
"""NOVA""",2012-09-02 17:35:00,"""Jak se budí princezny""","""29.12.2024 v 10:00 (JOJ+)29.12.2024 v 14:15 (NOVA)30.12.2024 v 10:45 (Nova Cinema)""","""Další díl/opakování:29.12.2024 v 10:00 (JOJ+)29.12.2024 v 14:15 (NOVA)30.12.2024 v 10:45 (Nova Cinema)""","""film/jak-se-budi-princezny/9284"""
"""Prima""",2014-12-20 14:10:00,"""Veselé Vánoce přejí chobotnice""",null,"""Komedie ČR (1986). Rodina Holanových je po návratu z dovolené bohatší o dvě malé chobotničky z prapodivné hmoty. Evička s Honzíkem mají kamarády, ale …","""film/vesele-vanoce-preji-chobotnice/22422"""


In [6]:
do_grafu = pl.DataFrame(
    {'datum':range(1,53)}
).join(
    df.filter(
        pl.col("kanal").is_in(["ČT1","ČT2","NOVA","Prima"])
    ).filter(
        pl.col("nazev").is_in(vanocni_filmy)
    ).group_by(
        ['nazev',pl.col('datum').dt.week()]
    ).len(),
    how='left',
    on='datum'
).fill_null(0).sort(by='datum')
    
do_grafu

datum,nazev,len
i64,str,u32
1,"""Dr. Dolittle 2""",1
1,"""Silvestr svobodného pána""",6
1,"""Sám doma""",1
1,"""Princezna ze mlejna""",2
1,"""Láska nebeská""",1
1,"""Kráska a zvíře""",2
1,"""Kotva u přívozu""",1
1,"""Hrátky s čertem""",2
1,"""Princ a Večernice""",2


In [7]:
do_grafu2 = df.filter(
        pl.col("kanal").is_in(["ČT1","ČT2","NOVA","Prima"])
    ).filter(
        pl.col("nazev").is_in(vanocni_filmy)
    ).with_columns(
        pl.col("nazev").str.replace("Dalskabáty, hříšná ves aneb Zapomenutý čert","Dalskabáty, hříšná ves").replace("Sám doma 2: Ztracen v New Yorku","Sám doma 2").replace("Veselé Vánoce přejí chobotnice","Veselé Ván. přejí chobotnice")
    ).filter(
        ((pl.col("datum").dt.month() == 12) & (pl.col("datum").dt.day() >= 22)) | ((pl.col("datum").dt.month() == 1) & (pl.col("datum").dt.day() <= 2))
    ).with_columns(
        (pl.col('datum').dt.month().cast(str) + pl.lit("-") + pl.col("datum").dt.day().cast(str)).str.replace('1-','13-').alias("kolikateho")
    ).with_columns(
        pl.when(pl.col('datum').dt.month() == 1).then(pl.col('datum').dt.day() + 31).otherwise(pl.col("datum").dt.day()).alias("kolikateho")
    ).group_by(
        ["nazev","kolikateho"]
    ).len(
    ).with_columns(
        (pl.when(
            pl.col('kolikateho') <= 31
        ).then(
            pl.col('kolikateho').cast(str) + pl.lit('.')
        ).otherwise(
            (pl.col('kolikateho') - 31).cast(str) + pl.lit('.')
        )).alias('kolikateho_str')
    )

poradi2 = do_grafu2.sort("len", descending=True).unique(subset="nazev",keep="first").with_columns(pl.lit('max').alias('maximum'))

poradi_dat = [f'{x}.' for x in [y for y in range(6,32)] + [y for y in range(1,6)]]

do_grafu2 = do_grafu2.join(poradi2, on=['nazev','kolikateho'], how='left').sort(by=['nazev','kolikateho']).fill_null(0).with_columns(
    pl.when(pl.col('maximum').is_null()).then(pl.lit('ne')).otherwise(pl.lit('max')).alias('maximum')
)

poradi2 = poradi2.sort(by=["kolikateho","len"],descending=[False,True]).select(pl.col("nazev")).to_series().to_list()

poradi3 = do_grafu2.group_by("nazev").agg(
    weighted_avg_kolikateho=(pl.col("kolikateho") * pl.col("len")).sum() / pl.col("len").sum()
).sort(
    by="weighted_avg_kolikateho"
).select(
    pl.col("nazev")
).to_series().to_list()

poradi3[0:5]

['Anděl Páně',
 'Grinch',
 'Chobotnice z II. patra',
 'Láska nebeská',
 'Cesta do Rokycan']

In [8]:
poradi_dat

['6.',
 '7.',
 '8.',
 '9.',
 '10.',
 '11.',
 '12.',
 '13.',
 '14.',
 '15.',
 '16.',
 '17.',
 '18.',
 '19.',
 '20.',
 '21.',
 '22.',
 '23.',
 '24.',
 '25.',
 '26.',
 '27.',
 '28.',
 '29.',
 '30.',
 '31.',
 '1.',
 '2.',
 '3.',
 '4.',
 '5.']

In [9]:
do_grafu2

nazev,kolikateho,len,kolikateho_str,len_right,kolikateho_str_right,maximum
str,i8,u32,str,u32,str,str
"""Anděl Páně""",22,1,"""22.""",0,null,"""ne"""
"""Anděl Páně""",23,13,"""23.""",13,"""23.""","""max"""
"""Anděl Páně""",25,1,"""25.""",0,null,"""ne"""
"""Ať přiletí čáp, královno!""",24,2,"""24.""",0,null,"""ne"""
"""Ať přiletí čáp, královno!""",25,1,"""25.""",0,null,"""ne"""
"""Ať přiletí čáp, královno!""",26,6,"""26.""",6,"""26.""","""max"""
"""Ať přiletí čáp, královno!""",31,1,"""31.""",0,null,"""ne"""
"""Ať přiletí čáp, královno!""",32,5,"""1.""",0,null,"""ne"""
"""Bohouš""",29,1,"""29.""",0,null,"""ne"""


In [10]:
df_cary = pl.DataFrame(
    {'kolikateho_str':['24.','31.'],'umisteni':poradi3[0]}
)
df_text = pl.DataFrame(
    {'kolikateho_str': ['27.'],'umisteni':poradi3[0]}
)

cary = alt.Chart(
    df_cary
).mark_rule(
    color='#A9AAC0',
    strokeWidth=16,
    opacity=0.3
).encode(
    alt.X('kolikateho_str:N',sort=poradi_dat)
)

text = alt.Chart(
    df_text
).mark_text(
    text=['← Štědrý den               Silvestr →'],
    color='black',#'#B25342',
    dx=10,
    dy=-409.5,
    baseline='middle',
    fontSize=9,
    fontWeight=300,
    font='Asap'
).encode(
    alt.X('kolikateho_str:N',sort=poradi_dat),
    # alt.Y('umisteni:N',sort=poradi3,title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5'))
)

(cary + text + alt.Chart(
    do_grafu2.to_pandas(),
    width=240,
    title=alt.Title(
        "Vánoční filmy a kdy typicky běží",
        subtitle=[
            'Snímky jsou řazené od těch vysílaných obvykle dříve po ty vysílané později.',
            'Velikost koleček odpovídá počtu repríz, nejčastější den reprízy je červený.'
        ]
    )
).mark_circle(
    opacity=1
).encode(
    alt.X(
        'kolikateho_str:N', # do_grafu2.sort(by='kolikateho').select(pl.col('kolikateho')).to_series().to_list(),
        sort=poradi_dat,
        title=None,
        axis=alt.Axis(orient='top',labelAngle=0,labelAlign='center',tickColor='white')
    ),
    alt.Y('nazev:N',sort=poradi3,title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5')),
    alt.Size('len:Q',legend=None),
    alt.Color("maximum:N",legend=None,scale=alt.Scale(range=['#B25342','#D6B599']))
)
).configure_view(
    stroke='transparent'
)

alt.LayerChart(...)

In [11]:
df.sample()

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""Prima LOVE""",2015-04-09 04:55:00,"""Vraždy v Kitzbühelu""","""20.01.2025 v 07:45 (Prima STAR)20.01.2025 v 09:35 (Prima Krimi)21.01.2025 v 07:45 (Prima STAR)21.01.2025 v 09:40 (Prima Krimi)22.01.2025 v 07:45 (Prim…","""Další díl/opakování:20.01.2025 v 07:45 (Prima STAR)20.01.2025 v 09:35 (Prima Krimi)21.01.2025 v 07:45 (Prima STAR)21.01.2025 v 09:40 (Prima Krimi)22.0…","""serial/vrazdy-v-kitzbuhelu-soko-kitzbuhel/45493"""


In [12]:
df.group_by("kanal").len().sort(by="len",descending=True)

kanal,len
cat,u32
"""ČT2""",248327
"""ČT1""",239798
"""Prima Cool""",170595
"""Prima""",163569
"""NOVA""",159461
"""Prima LOVE""",126488
"""Nova 2""",116882
"""TV Barrandov""",111391
"""Nova Cinema""",102404


In [187]:
def najdi_cloveka(clovek:str,hezky:str,kanaly=["ČT1","ČT2","NOVA","Prima","Prima Cool","Prima LOVE"]):
    return pl.DataFrame({'kolikateho':range(6,37)}).join(
        df.filter(
         pl.col("kanal").is_in(kanaly)
    ).filter(
        pl.col("extra").str.contains(clovek) | pl.col("noticka").str.contains(clovek) | pl.col("nazev").str.contains(clovek)
    ).filter(
        ((pl.col("datum").dt.month() == 12) & (pl.col("datum").dt.day() >= 6)) | ((pl.col("datum").dt.month() == 1) & (pl.col("datum").dt.day() <= 5))
    ).with_columns(
            pl.when(pl.col('datum').dt.month() == 1).then(pl.col('datum').dt.day() + 31).otherwise(pl.col("datum").dt.day()).alias("kolikateho")
    ).group_by(
        "kolikateho"
    ).len(),
    how='left',
    on='kolikateho').fill_null(0).sort(by="kolikateho").with_columns(
            (pl.when(
                pl.col('kolikateho') <= 31
            ).then(
                pl.col('kolikateho').cast(str) + pl.lit('.')
            ).otherwise(
                (pl.col('kolikateho') - 31).cast(str) + pl.lit('.')
            )).alias('kolikateho_str')
        ).with_columns(
        pl.lit(hezky).alias("kdo")
        )

In [15]:
najdi_cloveka('Hitler','Hitler')

kolikateho,len,kolikateho_str,kdo
i64,u32,str,str
6,2,"""6.""","""Hitler"""
7,1,"""7.""","""Hitler"""
8,3,"""8.""","""Hitler"""
9,2,"""9.""","""Hitler"""
10,4,"""10.""","""Hitler"""
11,1,"""11.""","""Hitler"""
12,0,"""12.""","""Hitler"""
13,2,"""13.""","""Hitler"""
14,2,"""14.""","""Hitler"""


In [47]:
def najdi_cloveka_celorok(clovek:str,hezky:str,kanaly=["ČT1","ČT2","NOVA","Prima","Prima Cool","Prima LOVE"]):
    return df.filter(
         pl.col("kanal").is_in(kanaly)
    ).filter(
        pl.col("extra").str.contains(clovek) | pl.col("noticka").str.contains(clovek)
    ).group_by(
        pl.col("datum").dt.month()
    ).len().sort(by="datum").with_columns(
        pl.lit(hezky).alias("kdo")
        )

In [17]:
do_grafu3 = pl.concat(
    [
        najdi_cloveka("Bohdalov","Bohdalová"),
        najdi_cloveka("Donutil","Donutil"),
        najdi_cloveka("Polívk","Polívka"),
        najdi_cloveka("Menšík","Menšík"),
        najdi_cloveka("Janžurov","Janžurová"),
        najdi_cloveka("Filipovs","Filipovský"),
        najdi_cloveka("Růžičkov","Růžičková"),
        najdi_cloveka("Kretschmerov","Kretschmerová"),
        najdi_cloveka("Šafránkov","Šafránková"),
    ]
)

alt.Chart(
    do_grafu3
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [18]:
do_grafu3 = pl.concat(
    [
        najdi_cloveka("komedi","komedie"),
        najdi_cloveka("drama","drama"),
        najdi_cloveka("zábavn","zábava"),
        najdi_cloveka("pohádk","pohádka"),
        najdi_cloveka("hudební","hudební"),
        najdi_cloveka("romantick","romantický"),
        najdi_cloveka("válečn","válečný"),
        najdi_cloveka("historick","historický"),
        najdi_cloveka("církevn","církevní"),
        najdi_cloveka("bibl","biblický"),
        najdi_cloveka("animovan","animovaný"),
        najdi_cloveka("detektiv","detektivní"),
    ]
)

alt.Chart(
    do_grafu3
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [19]:
vanocni.sort(by="nazev")

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64
"""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film""","""215414-andel-pane""","""Anděl Páně""","[""pohádka""]","""Česko""",2005,91,"[""Jiří Strach""]","[""Lucie Konášová""]","[""Ivan Trojan"", ""Jiří Dvořák"", … ""více""]","""Je časné ráno, v předvečer Štědrého dne, a celé nebe se chystá na velkolepé oslavy Ježíškových narozenin. Panna Marie (Klára Issová) se svatou Veronik…",77
"""https://www.csfd.cz/film/108859-at-prileti-cap-kralovno/prehled/""","""tv""","""108859-at-prileti-cap-kralovno""","""Ať přiletí čáp, královno!""","[""pohádka"", ""hudební""]","""Československo""",1988,93,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Ve volném pokračování pohádkové komedie Co takhle svatba, princi? se znovu setkáte s princem Davidem a jeho ženou Bělou. Oba se těší na děťátko, které…",61
"""https://www.csfd.cz/film/237811-bejvalek-se-nezbavis/prehled/""","""film""","""237811-bejvalek-se-nezbavis""","""Bejvalek se nezbavíš""","[""komedie"", ""romantický"", ""fantasy""]","""USA""",2009,96,"[""Mark Waters""]","[""Jon Lucas"", ""Scott Moore""]","[""Matthew McConaughey"", ""Jennifer Garner"", … ""více""]","""Své minulosti neutečete… Matthew McConaughey a Jennifer Garnerová zazáří v romantické komedii o nenapravitelném sukničkáři, kterému se jednoho dne zje…",57
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87
"""https://www.csfd.cz/film/187999-cesta-do-rokycan/prehled/""","""tv""","""187999-cesta-do-rokycan""","""Cesta do Rokycan""","[""komedie""]","""Československo""",1981,38,"[""Václav Hudeček""]","[""Miroslav Plzák""]","[""Iva Janžurová"", ""Vladimír Menšík"", ""Vladimír Dlouhý""]","""Vánoce jsou tradičně nazývány svátky štědrosti, pohody a míru. Zajistit slavnostní stůl, opatřit dárky a vyřešit, kdo bude pozván k štědrovečerní veče…",83
"""https://www.csfd.cz/film/7388-chobotnice-z-ii-patra/prehled/""","""tv""","""7388-chobotnice-z-ii-patra""","""Chobotnice z II. patra""","[""rodinný"", ""komedie"", ""sci-fi""]","""Československo / Západní Německo""",1986,92,"[""Jindřich Polák""]","[""Ota Hofman""]","[""Pavel Zedníček"", ""Dagmar Havlová"", … ""více""]","""Eva a Honzík si pro zábavu vymodelují z neznámé hmoty dvě chobotničky. Nic netuší, že hmota je živá a dokonce mluví. Kolem nich se začnou dít prapodiv…",66
"""https://www.csfd.cz/film/108854-co-takhle-svatba-princi/prehled/""","""tv""","""108854-co-takhle-svatba-princi""","""Co takhle svatba, princi?""","[""pohádka"", ""hudební""]","""Československo""",1986,90,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Kdo by neznal jednu z nejznámějších filmových písniček ""Když se načančám"". Tuto a další písničky (např. ""Taková jsem já"", ""Princezny jdou si hajnout"" …",69
null,null,"""3094-cisaruv-pekar-pekaruv-cisar""","""Císařův pekař - Pekařův císař""","[""komedie"", ""historický""]","""Československo""",1951,80,"[""Martin Frič""]","[""Jan Werich"", ""Martin Frič"",

In [20]:
vanocni.explode("hraji").group_by("hraji").len().sort(by="len",descending=True)

hraji,len
str,u32
"""více""",40
"""Jiřina Bohdalová""",10
"""Vladimír Menšík""",7
"""František Filipovský""",7
"""Petr Nárožný""",5
"""Helena Růžičková""",5
"""Jiří Lír""",5
"""Jaroslav Moučka""",5
"""Jaroslava Kretschmerová""",5


In [21]:
do_grafu4 = pl.concat(
    [
        najdi_cloveka_celorok("Bohdalov","Bohdalová"),
        najdi_cloveka_celorok("Donutil","Donutil"),
        najdi_cloveka_celorok("Polívk","Polívka"),
        najdi_cloveka_celorok("Menšík","Menšík"),
        najdi_cloveka_celorok("Janžurov","Janžurová"),
        najdi_cloveka_celorok("Filipovs","Filipovský"),
        najdi_cloveka_celorok("Růžičkov","Růžičková"),
        najdi_cloveka_celorok("Kretschmerov","Kretschmerová"),
        najdi_cloveka_celorok("Šafránkov","Šafránková")
    ]
)

alt.Chart(
    do_grafu4,
    width=600,
    height=50
).mark_bar(
).encode(
    alt.X("datum:Q",sort=poradi_dat),
    alt.Y("len:Q"),
    alt.Row("kdo:N")
)

alt.Chart(...)

In [55]:
do_grafu5 = pl.concat(
    [
        najdi_cloveka("Hitler","Hitler",kanaly=['ČT2']),
        najdi_cloveka("Ježíš","Ježíš",kanaly=['ČT2']),
    #    najdi_cloveka("Stalin","Stalin"),
    #    najdi_cloveka("Masaryk","Masaryk"),
    #    najdi_cloveka("Havel","Havel"),
    ]
)

alt.Chart(
    do_grafu5,
    width=600
).mark_circle(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat),
    alt.Y("kdo:N"),
    alt.Size("len:Q")
)

alt.Chart(...)

In [103]:
df.filter(pl.col("kanal") == "NOVA").select(pl.col("datum").max())

datum
datetime[μs]
2025-04-17 23:50:00


In [157]:
najdi_cloveka("(Ježíš|Kristu|Kristov)","Ježíš",kanaly=['ČT2'])

kolikateho,len,kolikateho_str,kdo
i64,u32,str,str
6,1,"""6.""","""Ježíš"""
7,1,"""7.""","""Ježíš"""
8,1,"""8.""","""Ježíš"""
9,0,"""9.""","""Ježíš"""
10,1,"""10.""","""Ježíš"""
11,0,"""11.""","""Ježíš"""
12,1,"""12.""","""Ježíš"""
13,0,"""13.""","""Ježíš"""
14,2,"""14.""","""Ježíš"""


In [211]:
do_grafu5 = pl.concat(
    [
        najdi_cloveka("(Hitler|Stalin|Mussolini)","Hitler, Stalin, Mussolini",kanaly=['ČT1','ČT2',"NOVA","Prima"]),
        najdi_cloveka("(Ježíš|Kristu|Kristov)","Ježíš",kanaly=['ČT1','ČT2',"NOVA","Prima"]),

    #    najdi_cloveka("Stalin","Stalin"),
    #    najdi_cloveka("Masaryk","Masaryk"),
    #    najdi_cloveka("Havel","Havel"),
    ]
)

alt.Chart(
    do_grafu5.filter((pl.col("kolikateho") <= 31) & (pl.col("kolikateho") >= 10)),
    title=alt.Title(
        "Týden, kdy Ježíš střídá Hitlera",
        subtitle=["Náměty pořadů vysílaných na ČT1, ČT2, Nově a Primě v prosincích 2007-24."]
    ),
    width=320,
    height=50
).mark_bar(
).encode(
    alt.X("kolikateho_str:N",sort=poradi_dat,title=None,axis=alt.Axis(grid=False, gridColor='#E5E5E5',labelAngle=0,labelAlign='center',tickColor='#E5E5E5')),
    alt.Y("len:Q",title=None,axis=alt.Axis(grid=True, gridColor='#E5E5E5',tickColor='#E5E5E5',labelExpr="datum.label + ' pořadů'", orient='right')),
    alt.Color("kdo:N",scale=alt.Scale(range=['#D6B599','#B25342']), legend=alt.Legend(orient='top',title=None))
).configure_view(
    stroke='transparent'
)

alt.Chart(...)

In [203]:
pl.DataFrame({'datum':range(1,366)}).join(
        df.filter(
         pl.col("kanal").is_in(['ČT1','ČT2',"NOVA","Prima"])
    ).filter(
        pl.col("extra").str.contains("(Hitler|Stalin|Mussolini)") | pl.col("noticka").str.contains("(Hitler|Stalin|Mussolini)") | pl.col("nazev").str.contains("(Hitler|Stalin|Mussolini)")
    ).group_by(
            pl.col("datum").dt.ordinal_day()
    ).len(),
    how="left",
    on="datum").filter(pl.col("len").is_null())

datum,len
i64,u32
1,null
224,null
359,null
365,null


In [217]:
df.filter(pl.col("nazev") == "Pelíšky").filter(pl.col("datum").dt.month() != 12).sort(by="datum")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2011-03-08 20:55:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2015-08-02 20:00:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""
"""ČT1""",2016-07-17 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2017-08-21 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2018-08-19 20:20:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2019-06-09 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2023-05-01 20:10:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesátých …","""film/pelisky/25517"""
"""ČT1""",2024-08-20 20:10:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesátých …","""film/pelisky/25517"""


In [215]:
df.filter(pl.col("nazev") == "Pelíšky").filter(pl.col("datum").dt.month() == 12).sort(by="datum")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2011-12-24 21:25:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2012-12-24 20:35:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2013-12-24 20:30:00,"""Pelíšky""",null,"""Film ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesátých let…","""film/pelisky/25517"""
"""ČT1""",2014-12-24 20:20:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""
"""ČT1""",2015-12-24 21:55:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""
"""ČT1""",2016-12-24 21:45:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2017-12-24 22:10:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2018-12-24 22:10:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
"""ČT1""",2019-12-24 22:05:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517"""
